In [61]:
# Import

path = "input/day7.txt"
file = open(path, 'r')
data = file.readlines()
file.close()

You are given a list of hands and their corresponding bid.
Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand.
(A new challenge in this problem is the idea of non-numeric value comparison. To address this easily, I plan to assign an arbitrary numeric value to each characteristic, and multiply the more important characteristics by larger values)

A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2 (descending order of strength).
Every hand is exactly one type. From strongest to weakest, they are:
- Five of a kind, where all five cards have the same label: AAAAA
- Four of a kind, where four cards have the same label and one card has a different label: AA8AA
- Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
- Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
- Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
- One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
- High card, where all cards' labels are distinct: 23456

In [62]:
from enum import Enum 
from collections import Counter

class HandType(Enum):
    FIVE_OF_KIND = 7
    FOUR_OF_KIND = 6
    FULL_HOUSE = 5
    THREE_OF_KIND = 4
    TWO_PAIR = 3
    ONE_PAIR = 2
    HIGH_CARD = 1

def map_char_to_val(char) -> int:
    match char:
        case "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9":
            return int(char)
        case "T":
            return 10
        case "J":
            return 11
        case "Q":
            return 12
        case "K":
            return 13
        case "A":
            return 14

def create_hand_val_dict(cards: str):
    # below converts 46644 to {4:3, 6:2} and sorts
    return dict(sorted(Counter(cards).items(), key=lambda x: (-x[1], -x[0])))

def find_hand_type(cards) -> HandType:
    if 5 in cards.values():
        return HandType.FIVE_OF_KIND
    elif 4 in cards.values():
        return HandType.FOUR_OF_KIND
    elif 3 in cards.values():
        if 2 in cards.values():
            return HandType.FULL_HOUSE
        else:
            return HandType.THREE_OF_KIND
    elif 2 in cards.values():
        if list(cards.values()).count(2) == 2:
            return HandType.TWO_PAIR
        else:
            return HandType.ONE_PAIR
    else:
        return HandType.HIGH_CARD
    

def assign_hand_score(cards, hand_type, card_array) -> float:
    score = hand_type.value * 100
    for i, key in enumerate(card_array):
        score += card_array[i] * (100 ** -i)
    return score

class Hand:
    def __init__(self, line: str) -> None:
        self.card_array = [map_char_to_val(x) for x in line.split()[0]]
        self.card_dict = create_hand_val_dict(self.card_array)
        self.hand_type = find_hand_type(self.card_dict)
        self.score = assign_hand_score(self.card_dict, self.hand_type, self.card_array)

        self.bet = int(line.split()[1])
    


In [63]:
# Prep data

hands = [Hand(line) for line in data]

Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand.
Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank.

In [64]:
# Part 1

# sort by hand rank
hands.sort(key=lambda x: x.score, reverse=False)

total_winnings = 0

for i, hand in enumerate(hands):
    total_winnings += (i + 1) * hand.bet

print(total_winnings)


253313241


Part 2

Cards designated J are now Jokers instead of Jacks. Jokers can be any value for purposes of finding the best hand type, but are the lowest card value for tie-breakers.

In [65]:
# Part 2 definition updates

def map_char_to_val(char) -> int:
    match char:
        case "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9":
            return int(char)
        case "T":
            return 10
        case "J":
            return 1 # updated from 11 to 1 for part 2
        case "Q":
            return 12
        case "K":
            return 13
        case "A":
            return 14

def find_hand_type(cards, joker_count) -> HandType:
    if joker_count == 5:
        largest_group = 0
    else:   
        largest_group = max(cards.values())
    if largest_group + joker_count == 5:
        return HandType.FIVE_OF_KIND
    elif largest_group + joker_count == 4:
        return HandType.FOUR_OF_KIND
    elif largest_group + joker_count == 3:
        if list(cards.values()).count(2) + list(cards.values()).count(3)  == 2:
            return HandType.FULL_HOUSE # normal FH (XXXYY) or 1 joker, 2 pair, (XXYYJ)
        else:
            return HandType.THREE_OF_KIND
    elif largest_group + joker_count == 2:
        if list(cards.values()).count(2) == 2:
            return HandType.TWO_PAIR # no joker would cause a 2 pair
        else:
            return HandType.ONE_PAIR
    else:
        return HandType.HIGH_CARD
    

def assign_hand_score(cards, hand_type, card_array) -> float:
    score = hand_type.value * 100
    for i, key in enumerate(card_array):
        score += card_array[i] * (100 ** -i)
    return score

class Hand:
    def __init__(self, line: str) -> None:
        self.card_array = [map_char_to_val(x) for x in line.split()[0]]
        self.card_dict = create_hand_val_dict(self.card_array)
        # remove jokers from dict and store as seperate value
        self.joker_count = self.card_dict.get(1)
        if self.joker_count == None:
            self.joker_count = 0
        self.card_dict.pop(1, None)

        self.hand_type = find_hand_type(self.card_dict, self.joker_count)
        self.score = assign_hand_score(self.card_dict, self.hand_type, self.card_array)

        self.bet = int(line.split()[1])

In [66]:
# Part 2 solve

hands = [Hand(line) for line in data]

# sort by hand rank
hands.sort(key=lambda x: x.score, reverse=False)

total_winnings = 0

for i, hand in enumerate(hands):
    total_winnings += (i + 1) * hand.bet

print(total_winnings)

253362743
